In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using Test
using SparseArrays
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
using ProfileView

Gtk-Message: 18:01:16.087: Failed to load module "canberra-gtk-module"
Gtk-Message: 18:01:16.087: Failed to load module "canberra-gtk-module"


In [52]:
Threads.nthreads()

10

### Integration

In [101]:
Nx = 20
Ny = 1
m = 20
Ne = 300
X = randn(Nx, Ne)#.*randn(Nx, Ne)# .+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [102]:
M = HermiteMap(m, X);

In [103]:
@time optimize(M, X, 10; P = thread)

  0.273911 seconds (913.47 k allocations: 1.021 GiB, 15.11% gc time)


HermiteMap(20, 20, LinearTransform(20, [-0.030340845081385105, -0.10480883822764896, -0.028334876990154134, 0.02413089298080643, -0.00667643985356715, 0.0516496425054286, -0.010447476384176747, 0.044235332639566784, 0.04067484086399342, -0.04458273843783829, -0.05165293554909299, -0.03903780137407225, -0.06646898110229327, 0.02789760737486254, -0.024947215991804154, -0.003173639900249363, 0.02610957064428385, -0.0031691028574343423, 0.07825489800067868, 0.05882955933979587], [0.9801933594913597 0.0 … 0.0 0.0; 0.0 1.0068487434976525 … 0.0 0.0; … ; 0.0 0.0 … 0.941047919064053 0.0; 0.0 0.0 … 0.0 1.0018322441332745], true), MapComponent[MapComponent(20, 10, 1, IntegratedFunction(20, 10, 1, Rectifier("softplus"), ParametricFunction(ExpandedFunction(20, 10, 1, MultiBasis(Basis of 20 functions: Constant -> 18th degree Probabilistic Hermite function
, 1), [1; 2; … ; 9; 10], [-0.0773109230779905, 0.6538513421737511, -0.9141798617940071, 3.2985210102365827, -2.7064061841905502, 6.744185226052195

In [104]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $5; P = thread)
    end
end

timing (generic function with 1 method)

In [106]:
timing();

  40.695 ms (446366 allocations: 333.25 MiB)


### Conditional sampling

In [362]:
Nx = 50
Ny = 20
m = 20
Ne = 500
Xprior = randn(Nx, Ne).*randn(Nx, Ne);
Xpost = deepcopy(Xprior) .+ 0.2*randn(Nx, Ne);

In [363]:
M = HermiteMap(m, Xprior; diag = true);

In [364]:
Ystar = deepcopy(Xpost[1:Ny,:]) + 0.05*randn(Ny,Ne);

In [365]:
@time M = HermiteMap(m, Xprior);
@time optimize(M, Xprior, 6; withconstant = false, start = Ny+1);

  0.000053 seconds (334 allocations: 28.531 KiB)
  1.402635 seconds (4.45 M allocations: 2.118 GiB, 6.45% gc time)


In [366]:
F = evaluate(M, Xpost; start = Ny+1)

50×500 Array{Float64,2}:
  0.0          0.0         0.0        …   0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0        …   0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0        …   0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  0.0          0.0         0.0            0.0        0.0          0.0
  ⋮                                   ⋱                          

In [367]:
@time inverse!(F, M, Xprior, Ystar; start = Ny+1)

  0.017258 seconds (14.27 k allocations: 31.560 MiB, 30.98% gc time)


50×500 Array{Float64,2}:
 -0.211342    0.81505     -0.0197552  …   0.135453     1.29042
 -0.59309     0.114505     0.531372       4.28245     -0.275798
 -0.238622   -0.324534    -0.187113       1.13681     -0.139542
  0.39385     0.180453     1.12909       -1.49309     -0.0112934
 -0.274126    0.3228      -0.0252479      0.188278     0.425514
 -0.133536   -1.11297     -0.719724   …   0.213793     0.161787
  0.512472    0.697728     1.07623       -0.255019     0.0535651
 -0.117983   -0.30399     -0.191594       2.05243      0.25097
 -0.89177     0.15308     -2.1008        -1.15588      0.00267544
 -0.043514   -0.149709     1.16827        0.0679464    0.968786
  3.74337     0.213884     1.26443    …  -0.470425    -0.653102
 -0.569208    1.05778     -0.390264       0.00269174  -0.0421446
  0.709907   -1.02491     -0.302788       0.381497     0.137157
  ⋮                                   ⋱               
  2.26296    -0.279945     0.227358      -0.0597177   -0.533637
 -1.11994     0.66640

In [368]:
Xprior

50×500 Array{Float64,2}:
 -0.211342    0.81505     -0.0197552  …   0.135453     1.29042
 -0.59309     0.114505     0.531372       4.28245     -0.275798
 -0.238622   -0.324534    -0.187113       1.13681     -0.139542
  0.39385     0.180453     1.12909       -1.49309     -0.0112934
 -0.274126    0.3228      -0.0252479      0.188278     0.425514
 -0.133536   -1.11297     -0.719724   …   0.213793     0.161787
  0.512472    0.697728     1.07623       -0.255019     0.0535651
 -0.117983   -0.30399     -0.191594       2.05243      0.25097
 -0.89177     0.15308     -2.1008        -1.15588      0.00267544
 -0.043514   -0.149709     1.16827        0.0679464    0.968786
  3.74337     0.213884     1.26443    …  -0.470425    -0.653102
 -0.569208    1.05778     -0.390264       0.00269174  -0.0421446
  0.709907   -1.02491     -0.302788       0.381497     0.137157
  ⋮                                   ⋱               
  2.26296    -0.279945     0.227358      -0.0597177   -0.533637
 -1.11994     0.66640

In [369]:
Ystar

20×500 Array{Float64,2}:
 -0.211342   0.81505    -0.0197552   0.936283    …   0.135453     1.29042
 -0.59309    0.114505    0.531372   -1.26934         4.28245     -0.275798
 -0.238622  -0.324534   -0.187113   -0.546287        1.13681     -0.139542
  0.39385    0.180453    1.12909    -0.346343       -1.49309     -0.0112934
 -0.274126   0.3228     -0.0252479  -1.91159         0.188278     0.425514
 -0.133536  -1.11297    -0.719724    0.00864939  …   0.213793     0.161787
  0.512472   0.697728    1.07623    -0.345924       -0.255019     0.0535651
 -0.117983  -0.30399    -0.191594    0.385232        2.05243      0.25097
 -0.89177    0.15308    -2.1008      0.352988       -1.15588      0.00267544
 -0.043514  -0.149709    1.16827     0.938497        0.0679464    0.968786
  3.74337    0.213884    1.26443    -1.5657      …  -0.470425    -0.653102
 -0.569208   1.05778    -0.390264   -0.123471        0.00269174  -0.0421446
  0.709907  -1.02491    -0.302788    0.730333        0.381497     0.1371

In [370]:
evaluate(M, Xpost)

50×500 Array{Float64,2}:
 -0.214688     1.02791     0.0675242   1.23873    …   0.246486     1.50265
 -0.625352     0.102629    0.496908   -1.24073        4.30769     -0.367575
 -0.317337    -0.402729   -0.209315   -0.626494       1.0512      -0.207429
  0.405501     0.154591    1.0362     -0.341916      -1.44798     -0.017184
 -0.254447     0.372489   -0.0585518  -1.8341         0.141994     0.486907
  0.016819    -0.951504   -0.554554    0.0937045  …   0.217138     0.165238
  0.547669     0.727836    1.14979    -0.420044      -0.339085    -0.0804486
 -0.148099    -0.300201   -0.175427    0.454183       2.16016      0.333894
 -0.95656      0.143077   -2.1566      0.284568      -1.21413      0.0524843
  0.091407    -0.0811638   1.31727     0.983804       0.149406     1.10267
  3.61501      0.346439    1.31026    -1.33758    …  -0.239404    -0.551078
 -0.594817     1.08567    -0.373397   -0.195372      -0.0162946   -0.0750069
  0.811885    -1.07275    -0.266239    0.811285       0.374198

In [371]:
100*norm(evaluate(M, Xprior; start = Ny+1)-evaluate(M, Xpost; start = Ny+1))/norm(evaluate(M, Xpost; start = Ny+1))

3.942579896291548e-6

In [373]:
100*norm(Xprior[Ny+1:end,:] - Xpost[Ny+1:end,:])/norm(Xpost[Ny+1:end,:])

0.7877296833552324